In [1]:
#Importando Bibliotecas
import pandas as pd
from datetime import timedelta
import scipy.stats as stats

In [2]:
#Importando as Bases
df_full = pd.read_csv('5.SERV_FALEC_EXON.csv', sep=';', encoding='UTF-8').assign(IDADE = '')

df_full['DT_NASC_SERVIDOR'] = pd.to_datetime(df_full['DT_NASC_SERVIDOR'], yearfirst=True, format='%d/%m/%Y')
df_full['DT_BASE'] = '31/12/2020'
df_full['DT_BASE'] = pd.to_datetime(df_full.DT_BASE, yearfirst=True, format='%d/%m/%Y')
df_full['IDADE'] = round((df_full.DT_BASE - df_full.DT_NASC_SERVIDOR) / timedelta(days=365.25) - 0.5).astype(int)

In [3]:
#Importando as Tábuas
df_tabuas = pd.read_excel('Funções Biométricas (Com Fórmulas).xlsx', sheet_name='qx')

In [114]:
ary_expostos = []
ary_esperados = []
for l in range(126+1):
    linha = []
    linha2 = []
    #Criando uma matriz com a quantidade de registros de cada IDADE
    for c in range(1):
        qtda = len(df_full[df_full['IDADE'] == l]) #Quantidade de eventos ocorridos em cada idade
        linha.append(qtda)
    ary_expostos.append(linha)

    #Criando uma matriz com o número de MORTES esperadas por idade e o X² calculado
    for c2 in range(len(df_tabuas.columns)):
        esperado = ary_expostos[l][0] * df_tabuas.iloc[l, c2] #Expostos * qx da tábua
        linha2.append(esperado)
    ary_esperados.append(linha2)

df_expostos = pd.DataFrame(data=ary_expostos)
df_esperados = pd.DataFrame(data=ary_esperados, columns=df_tabuas.columns.values)

In [115]:
#Cálculo do X²
ary_qui_quadrado = []
for l3 in range(127):
    linha3 = []
    for c3 in range(len(df_tabuas.columns)):
        if df_esperados.iloc[l3, c3] == 0.0:
            qui_quadrado = 0
            linha3.append(qui_quadrado)
        elif df_esperados.iloc[l3, c3] != 0.0:
            qui_quadrado = (ary_expostos[l3][0] - df_esperados.iloc[l3, c3])**2/df_esperados.iloc[l3, c3]
            linha3.append(qui_quadrado)
    ary_qui_quadrado.append(linha3)

df_qui_quadrado = pd.DataFrame(data=ary_qui_quadrado, columns=df_tabuas.columns.values)
df_qui_quadrado.loc['qui_quadrado'] = round(df_qui_quadrado.sum(axis=0), 2)
df_qui_quadrado.loc['p_value'] = 1-stats.chi2.cdf(x=df_qui_quadrado.loc['qui_quadrado'], df = int(df_expostos[df_expostos != 0].count() - 1))
qui_quadrado_tabu = stats.chi2.ppf(q = 0.95, df = int(df_expostos[df_expostos != 0].count() - 1))

In [145]:
#Cálculo do Kolmogorov-Smirnov K_S

df_esperados_acum = df_esperados.cumsum()
df_expostos_acum = df_expostos.cumsum()
ary_k_s = []
for l in range(126+1):
    linha2 = []
    for c2 in range(len(df_tabuas.columns)):
        k_s_calculado =  round(abs(((float(df_expostos_acum.loc[l]) / int(df_expostos.sum()))) - (float(df_esperados_acum.iloc[l, c2]) / df_esperados.iloc[:,c2].sum())), 6)
        linha2.append(k_s_calculado)
    ary_k_s.append(linha2)

df_k_s = pd.DataFrame(data=ary_k_s, columns=df_tabuas.columns.values)
df_k_s.loc['k_s'] = df_k_s.max(axis=0)
df_k_s.loc['p_value'] = stats.chi2.cdf(x=df_k_s.loc['k_s'], df = int(df_expostos[df_expostos != 0].count()))
df_k_s.tail()

,ALLG-72,AMERICAN EXPERIENCE,AT2000 (Suavizada 10%)_FEM,AT2000 (Suavizada 10%)_MAS,AT-2000 FEMALE,AT-2000 MALE,AT-49 FEMALE,AT-49 MALE,AT-50,AT-55,...,RRB-1944 Mod - Masc,TASA-1927,WYATT 1985,ZIMMERMANN EMPR. ESCRIT,Atuários Ingleses,Hubbard Laffitte,J.P. Jansen,Kinkelin,Moser,Samuel Dumas
124,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
125,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
126,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
k_s,0.0,2.931700e-01,4.809130e-01,4.766320e-01,4.811570e-01,4.764090e-01,4.814830e-01,5.033970e-01,5.079580e-01,5.005000e-01,...,4.755310e-01,8.874280e-01,8.463710e-01,6.294760e-01,1.603110e-01,1.529530e-01,1.958860e-01,1.926430e-01,2.119030e-01,1.534380e-01
p_value,0.0,2.432499e-51,1.413575e-45,1.112668e-45,1.432900e-45,1.098815e-45,1.459115e-45,4.801791e-45,6.112409e-45,4.114652e-45,...,1.045871e-45,1.774323e-38,5.036995e-39,1.887398e-42,2.165962e-58,6.112906e-59,4.767433e-56,3.042350e-56,3.949492e-55,6.656857e-59


In [140]:
import numpy as np
import pandas as pd
import scipy.stats as stats

'''ary = np.array([[6,1], [1.409100,1.523520]])
chi2_contingency(ary)

print(chi2_contingency(ary))

#15,137217'''

#qui_quadrado_tabu = stats.chi2.ppf(q = 0.95, df = int(df_expostos[df_expostos != 0].count() - 1))
#p_valor = 1-stats.chi2.cdf(x=18.194805, df=4)
#p_valor = round(1-stats.chi2.cdf(x=0.060, df= 19), 6)

#p_valor_k_s = stats.kstest(df_k_s['AMERICAN EXPERIENCE'], 'norm')
p_valor_k_s = stats.chi2.cdf(x=df_k_s.loc['SOMA'], df = int(df_expostos[df_expostos != 0].count() - 1))

print('Critical Value')
#print(qui_quadrado_tabu)
#print(round(p_valor_k_s, 6))
print(p_valor_k_s)

Critical Value
[0.00000000e+00 2.79295169e-12 7.87949597e-08 6.50206532e-08
 7.91172797e-08 6.34119468e-08 1.08048554e-07 1.33109637e-07
 1.76086333e-07 1.92548731e-07 8.90799860e-08 8.15881478e-08
 8.06766596e-08 6.73678119e-08 6.80643229e-08 3.28422233e-08
 5.07663083e-09 5.43923178e-08 6.34385682e-09 2.73215214e-08
 1.08340479e-08 4.79859577e-08 2.52271727e-08 7.97039555e-08
 2.74053559e-08 9.49067450e-08 3.59146364e-08 2.42310888e-08
 3.68723753e-09 3.83193240e-08 2.04734587e-08 1.75247900e-08
 4.16608371e-08 3.83193240e-08 2.08933852e-07 2.51094176e-07
 3.04517423e-08 9.48614372e-08 2.57660298e-07 3.77438433e-07
 2.76311450e-07 2.76568891e-07 4.07198761e-07 4.06844726e-07
 4.54586368e-07 2.76376266e-07 2.98916466e-08 1.34121900e-07
 2.18689549e-07 6.79542506e-08 4.36475989e-09 4.44869136e-10
 3.24986320e-09 7.20746594e-12 1.00463400e-09 1.12599203e-09
 1.29515884e-09 1.49358630e-09 6.21534433e-07 7.63514904e-12
 1.08399196e-06 6.05184860e-07 2.79820285e-08 8.49876900e-08
 7.041061

In [100]:
df_k_s['AMERICAN EXPERIENCE'].sum()

131336.0